### Label propagation using CNN feature extraction 

In [21]:
from data_processor import train_loader, test_loader, unlabeled_loader
import torch
import torch.nn as nn
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.semi_supervised import LabelSpreading
from sklearn.semi_supervised import LabelPropagation
from models import CNNet

In [2]:
model = CNNet()
model = torch.load('trained_models/trained_cnn')

/home/khalifa/.local/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'models.CNNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [15]:
with torch.no_grad(): 
    batch = next(iter(train_loader))
    images, labels = batch
    all_out = model.get_features(images)
    all_labels = labels 
    
    first_batch = len(images)
    
    for batch in train_loader:
        images, labels = batch 
        all_out = torch.cat((all_out, model.get_features(images)), 0)
        all_labels = torch.cat((all_labels, labels), 0)

    for batch in unlabeled_loader:
        images, labels = batch
        all_out = torch.cat((all_out, model.get_features(images)), 0)
        unlabel = torch.tensor([-1]*len(labels))
        all_labels = torch.cat((all_labels, unlabel), 0)

all_out = all_out[first_batch:]
all_labels = all_labels[first_batch:]

In [16]:
print(all_out.shape)
print(all_labels.shape)

torch.Size([14809, 50])
torch.Size([14809])


In [17]:
test_images, test_labels = next(iter(test_loader))
test_features = model.get_features(test_images)

with torch.no_grad(): 
    for images, labels in test_loader:
        test_features = torch.cat((test_features, model.get_features(images)), 0)
        test_labels = torch.cat((test_labels, labels), 0)

test_features = test_features[first_batch:]
test_labels = test_labels[first_batch:]

In [18]:
print(test_features.shape)
print(test_labels.shape)

torch.Size([1400, 50])
torch.Size([1400])


In [22]:
label_spr_model = LabelSpreading(max_iter = 50, kernel = 'knn', alpha = 0.01, n_neighbors = 100, tol = .001)

label_spr_model.fit(all_out.numpy(), all_labels.numpy())

label_spr_model.score(test_features.numpy(), test_labels.numpy())

0.9035714285714286